# Building Effective Agents

This is based on [this article](https://www.anthropic.com/engineering/building-effective-agents) by [Anthropic](https://www.anthropic.com/). We will explore common patterns for building effective LLM agentic systems using pure Python around LLM APIs. In particular, we use the [Python SDK](https://github.com/openai/openai-python/tree/main) for the [OpenAI API](https://platform.openai.com/docs/api-reference/introduction). 

## OpenAI API client

First, we need to load the API key in the environmental variables. The client expects the variable name `OPENAI_API_KEY` which we load from the `.env` file. This is easy to implement:

In [1]:
import inspect
from notebooks.utils import load_dotenv
print(inspect.getsource(load_dotenv))

def load_dotenv(verbose=False):
    with open(".env") as f:
        for line in f.readlines():
            k, v = line.split("=")
            os.environ[k] = v.strip().strip('"')
            if verbose:
                print(f"Loaded env variable: {k}")



In [2]:
load_dotenv(verbose=True)

Loaded env variable: OPENAI_API_KEY


Then the API key is automatically read by the **client**:

In [ ]:
from openai import OpenAI

client = OpenAI()

response = client.responses.create(
  model="gpt-4.1",
  input="Tell me a three sentence bedtime story about a unicorn."
)

In [ ]:
#| code-fold: true
import textwrap
text = response.output[0].content[0].text
wrapped_lines = textwrap.wrap(text, width=80)
[print(line) for line in wrapped_lines];

Luna the unicorn loved to gallop beneath the shimmering stars, her silver horn
glowing softly in the moonlight. One magical night, she discovered a hidden
grove where fireflies danced and flowers sang lullabies just for her. As she
curled up to rest, the peaceful melodies wrapped around her like a cozy blanket,
carrying her into the sweetest dreams.


Entire model response:

In [ ]:
from pprint import pprint
pprint(response.model_dump())

{'background': False,
 'conversation': None,
 'created_at': 1755889803.0,
 'error': None,
 'id': 'resp_68a8c08bac0881a1bc692232f86ac58407baeab058dc36b4',
 'incomplete_details': None,
 'instructions': None,
 'max_output_tokens': None,
 'max_tool_calls': None,
 'metadata': {},
 'model': 'gpt-4.1-2025-04-14',
 'object': 'response',
 'output': [{'content': [{'annotations': [],
                          'logprobs': [],
                          'text': 'Luna the unicorn loved to gallop beneath '
                                  'the shimmering stars, her silver horn '
                                  'glowing softly in the moonlight. One '
                                  'magical night, she discovered a hidden '
                                  'grove where fireflies danced and flowers '
                                  'sang lullabies just for her. As she curled '
                                  'up to rest, the peaceful melodies wrapped '
                                  'around 

## Building block: The augmented LLM

This is the foundational building block of agentic systems. The **augmented LLM** is a language model enhanced with **retrieval**, **tools**, and **memory**. Current models, due to their reasoning and understanding capabilities, are able to effectively generate their own search queries, select appropriate tools, and determinine what information to retain to solve problems or perform tasks.

![Augmented LLM](../img/augmented-llm.png)

In [ ]:
import json
from openai import OpenAI
from pydantic import BaseModel

class RetainResponse(BaseModel):
    retain: bool
    key: str
    value: str
    reason: str


client = OpenAI()
memory_file = "agent_memory.json"

def load_memory():
    try:
        return json.load(open(memory_file))
    except FileNotFoundError:
        return {}
    
def reset_memory():
    with open(memory_file, "w") as f:
        json.dump({}, f, indent=2)

def save_memory(memory):
    with open(memory_file, "w") as f:
        json.dump(memory, f, indent=2)

def agent(user_input):
    memory = load_memory()
    system_prompt = (
        f"You have access to a memory store: {memory}. "
        "Decide what new information should be remembered. "
        "NOTE: The key for a given value should be fairly generic."
    )
    
    completion = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ],
        response_format=RetainResponse,
    )

    response = completion.choices[0].message.parsed
    return response


def action(user_input: str):
    response = agent(user_input)
    if response.retain:
        memory = load_memory()
        values = set(memory.get(response.key, []))
        values.add(response.value)
        memory[response.key] = list(values)
        save_memory(memory)
    return response

In [ ]:
import pandas as pd

inputs = [
    "Hi, my name is Ron. Please remember that.",
    "I'm Doug. I use this application frequently.",
    "My name's Joe. I'm a temporary user.",
    "I'm Q. I used the app today.",
    "My name's Karen. I don't want any of my information in your systems."
]

df_resp = pd.DataFrame([action(text).model_dump() for text in inputs])

Results:

In [ ]:
#| echo: false
import warnings
warnings.simplefilter("ignore")
pd.set_option('display.max_colwidth', None)

df_resp.loc[0, "retain"] = "True⠀⠀⠀⠀⠀" if df_resp.loc[0, "retain"] else "False⠀⠀⠀⠀⠀"
df_resp.loc[0, "value"] = df_resp.loc[0, "value"] + "⠀⠀⠀⠀⠀"
df_resp.loc[0, "key"]   = df_resp.loc[0, "key"] + "⠀⠀⠀⠀"
df_resp[["key", "value", "retain", "reason"]]

,key,value,retain,reason
0,user_name⠀⠀⠀⠀⠀,Ron⠀⠀⠀⠀⠀⠀⠀⠀,True⠀⠀⠀⠀,Knowing the user's name helps personalize interactions in future conversations.
1,user_name,Doug,True,"The user identified themselves as Doug and mentioned frequent usage of the application, which implies that remembering this information would enhance user interaction and personalization."
2,user_name,Joe,False,"Since the user has indicated that they are a temporary user, it might not be necessary to retain their name for long-term purposes."
3,user_name,Q,True,A new user's name has been introduced and should be retained in the memory store for future reference.
4,user_name,Karen,False,The user explicitly requested not to have their information stored in the system.


In [ ]:
print(load_memory())
reset_memory()

{'user_name': ['Doug', 'Q', 'Ron']}
